# Calculate energy percentages per geo

- This notebook feches data about the energy consumption per Swedish municipality from the SCB API.
- The data is patchy but covers 2009-2022.
- We add 4 aggregations, summing over 4 groups of municipalities (kommunförbund), to this data.
- We then calculate the energy used in each municipality as a percentage of the total (the region VGR).
- In the current version we use only data from 2022 as that year has data from every municipality.

In [1]:
# Import libraries

import sys
import logging
import pandas as pd
import requests
import json
from io import StringIO
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent.parent
sys.path.append(str(root_path))

import paths

logging.basicConfig(level=logging.INFO)

In [2]:
# Load the data from SCB API

url = "https://api.scb.se/OV0104/v1/doris/sv/ssd/START/EN/EN0203/EN0203A/SlutAnvSektor"
query = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "item",
        "values": [
          "14", "1401", "1402", "1407", "1415", "1419", "1421", "1427", "1430", "1435",
          "1438", "1439", "1440", "1441", "1442", "1443", "1444", "1445", "1446", "1447",
          "1452", "1460", "1461", "1462", "1463", "1465", "1466", "1470", "1471", "1472",
          "1473", "1480", "1481", "1482", "1484", "1485", "1486", "1487", "1488", "1489",
          "1490", "1491", "1492", "1493", "1494", "1495", "1496", "1497", "1498", "1499"
        ]
      }
    },
    {
      "code": "Forbrukningskategri",
      "selection": {
        "filter": "item",
        "values": ["999"]
      }
    },
    {
      "code": "Bransle",
      "selection": {
        "filter": "item",
        "values": ["955"]
      }
    }
  ],
  "response": {
    "format": "csv"
  }
}

##  Make the request (POST)
response = requests.post(url, json=query)
if response.status_code == 200:
    csv_data = StringIO(response.text)
    response_csv = pd.read_csv(csv_data)    
else:
    print(f"Error: {response.status_code}")

## Format dataframe
energy_data = response_csv.copy()
energy_data['Kod'] = energy_data['region'].apply(lambda x: x.split(' ', 1)[0])
energy_data.set_index('Kod', inplace=True)
energy_data['region'] = energy_data['region'].apply(lambda x: x.split(' ', 1)[1])

In [3]:
# Build kommunförbunden (only the 4 in VGR for now)

geo_config_path = paths.geo_root / 'config.json'

with open(geo_config_path, "r") as f:
    config = json.load(f)

sections = config['weather-geo']['Västra Götalands län']['sections']

groups = {key: value for key, value in sections.items() if isinstance(value, list)}

In [4]:
# Add kommunforbund by summing up municipalities
for region, codes in groups.items():
    selection = energy_data.loc[codes]
    summed_selection = selection.iloc[:, 3:].apply(pd.to_numeric, errors='coerce').sum()
    new_row = pd.Series([region, 'totalt', 'totalt'] + summed_selection.tolist(), index=energy_data.columns)
    kod = '-'.join(codes)
    energy_data.loc[kod] = new_row

In [5]:
# Calculate percentages as energy per geo divided by total (VGR=14)

energy_percentages = pd.DataFrame(energy_data['Slutanvändning (MWh) 2022'] / energy_data.loc['14', 'Slutanvändning (MWh) 2022'])
energy_percentages.rename(columns={'Slutanvändning (MWh) 2022': 'percentage'}, inplace=True)

In [6]:
# Write the file

energy_percentages.to_csv(paths.demand_root / 'energy-percentages-vgr.csv.gz', compression='gzip')